<a href="https://colab.research.google.com/github/lucasichen/chem-challenge/blob/main/chem.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [212]:
import tensorflow as tf
import tensorflow_datasets as tfds
from tensorflow import keras
import matplotlib.pyplot as plt
import numpy as np
from tensorflow.keras import datasets, layers, models
from sklearn.neural_network import MLPClassifier
import pandas as pd
from transformers import AutoTokenizer
import pickle

tokenizer = AutoTokenizer.from_pretrained("seyonec/ChemBERTa-zinc-base-v1") 

In [214]:
# Get raw data from challenge
df = pd.read_csv('https://raw.githubusercontent.com/magarveylab/ec_chem_machacks2/main/datasets/reaction_train_dataset.csv')

In [215]:
# Visualize data to row 5
df.head(5)

,reaction_id,reaction_smiles,mapped_reaction_smiles,ec_level_1,ec_level_2,ec_level_3,ec_level_4
0,28330,COS(=O)(=O)[O-].C[n+]1c2ccccc2nc2ccccc21.NCCCN...,[n:1]1[c:2]2[cH:3][cH:4][cH:5][cH:6][c:7]2[n+:...,1,1.5,1.5.99,1.5.99.6
1,6697,C=CC1=C(C)c2cc3[nH]c(cc4nc(cc5[nH]c(cc1n2)c(C)...,[Fe+3:97].[Fe+3:98].[O:1]=[C:2]([CH3:3])[CH2:4...,1,1.3,1.3.2,1.3.2.3
2,33621,CC(=O)SCCNC(=O)CCNC(=O)[C@H](O)C(C)(C)COP(=O)(...,[CH3:10][C:21](=[O:40])[S:60][CH2:16][CH2:15][...,2,2.3,2.3.1,2.3.1.5
3,37450,Nc1ccccc1C(=O)C[C@H](N)C(=O)O.O=C(O)CCCC(=O)C(...,[CH2:1]([CH2:2][C:4]([C:6](=[O:10])[OH:11])=[O...,2,2.6,2.6.1,2.6.1.7
4,38912,CCCCCCCCC=CCCCCCCCC(=O)OCC(COP(=O)(O)OCCN)OC(=...,[CH3:1][CH2:3][CH2:5][CH2:7][CH2:9][CH2:11][CH...,3,3.1,3.1.4,3.1.4.4


In [216]:
# Definitly not necessary to copy
df_copy.head()
y = df_copy.drop(["reaction_smiles"], axis=1)
X = df_copy.drop(["ec_level_1"], axis=1)
# testing purpose
print(y['ec_level_1'].values)
print(X['reaction_smiles'].values[0])


[1 1 2 ... 2 4 2]
COS(=O)(=O)[O-].C[n+]1c2ccccc2nc2ccccc21.NCCCNCCCCNCCCN>>CN1c2ccccc2Nc2ccccc21.COS(=O)(=O)O.NCCC=O.NCCCCNCCCN


In [210]:
## IGNORE was for testing padding for different size arrays

# arr = tokenizer.encode_plus(X["reaction_smiles"].values[7]).input_ids
# test = (1713-len(arr))
# print(len(arr))
# arr_len = (int(test/2))
# arr1 = np.pad(arr, (arr_len,), 'constant')
# print(len(arr1))
# if(len(arr) != 1713) :
#   np.append(arr,[0],axis=0)
# print(arr)
# print(len(arr))

188
1712
[0, 39, 19, 39, 12, 39, 295, 39, 19, 39, 263, 51, 13, 666, 263, 51, 13, 332, 263, 51, 272, 39, 36, 44, 265, 51, 13, 39, 12, 39, 281, 39, 13, 647, 263, 51, 281, 51, 13, 634, 263, 51, 281, 51, 13, 279, 63, 39, 36, 44, 65, 21, 51, 63, 39, 266, 44, 265, 82, 22, 316, 23, 71, 12, 50, 13, 355, 333, 272, 39, 36, 44, 265, 51, 272, 39, 266, 44, 65, 21, 634, 263, 51, 281, 51, 13, 51, 18, 322, 263, 51, 13, 51, 34, 34, 39, 19, 39, 12, 39, 295, 39, 64, 39, 263, 51, 13, 322, 263, 51, 13, 51, 18, 262, 12, 39, 281, 647, 263, 51, 281, 51, 13, 634, 263, 51, 281, 51, 13, 279, 63, 39, 36, 44, 65, 21, 51, 63, 39, 266, 44, 265, 82, 22, 316, 23, 71, 12, 50, 13, 355, 333, 272, 39, 36, 44, 265, 51, 272, 39, 266, 44, 65, 21, 634, 263, 51, 281, 51, 13, 51, 272, 39, 266, 44, 265, 51, 13, 39, 263, 51, 13, 366, 263, 51, 13, 459, 2]
188


In [217]:
# Create training_data array eg.[[numerical-format-of-reaction],ec_value]

for i in range(len(X)):
   try:
     new_array = tokenizer.encode_plus(X["reaction_smiles"].values[i]).input_ids
     training_data.append([new_array,y["ec_level_1"].values[i]])
   except Exception as e:
      pass
# training_data
# def create_training_data():
#   for category in CATEGORIES:
#     print(category)
#     for smiles in X:
#       try:
#         new_array = (tokenizer.encode_plus(smiles).input_ids)
#         training_data.append([new_array,class_num])
#       except Exception as e:
#         print("error")
# create_training_data()
# print(len(training_data))
# training_data

Token indices sequence length is longer than the specified maximum sequence length for this model (584 > 512). Running this sequence through the model will result in indexing errors


In [218]:
X = []
y = []
training_data[0]
for features, label in training_data:
  X.append(features)
  y.append(label)
X = np.array(X)
y = np.array(y)

# To save data across, not sure how it works
pickle_out = open("X.pickle","wb")
pickle.dump(X, pickle_out)
pickle_out.close()

pickle_out = open("y.pickle","wb")
pickle.dump(y,pickle_out)
pickle_out.close()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:9: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  if __name__ == '__main__':


In [219]:
X = pickle.load(open("X.pickle","rb"))
y = pickle.load(open("y.pickle","rb"))

In [220]:
# print(type(X))

# Create new DataFrame with Smiles and ec_level
df = pd.DataFrame(X, columns = ['smiles'])
df['ec_level_1'] = y
# print(df)
# print(type(df))
y = df['ec_level_1']
X = df['smiles']
df.head(10)

,smiles,ec_level_1
0,"[0, 691, 263, 51, 287, 51, 272, 51, 288, 18, 3...",1
1,"[0, 39, 33, 262, 21, 33, 39, 12, 39, 13, 71, 2...",1
2,"[0, 262, 263, 51, 13, 666, 263, 51, 13, 332, 2...",2
3,"[0, 277, 21, 269, 21, 39, 263, 51, 13, 39, 63,...",2
4,"[0, 285, 305, 33, 285, 305, 263, 51, 13, 323, ...",3
5,"[0, 14, 18, 14, 50, 63, 39, 266, 44, 265, 297,...",2
6,"[0, 51, 33, 39, 12, 51, 13, 39, 263, 51, 13, 2...",4
7,"[0, 39, 19, 39, 12, 39, 295, 39, 19, 39, 263, ...",2
8,"[0, 277, 21, 270, 22, 71, 12, 371, 22, 63, 39,...",4
9,"[0, 14, 55, 63, 42, 73, 65, 21, 12, 55, 63, 14...",1


In [221]:
# Get max len of a smile array
df["smiles"].apply(lambda x: len(x)).max()

1713

In [222]:
clf = MLPClassifier(solver='lbfgs', alpha=1e-5,hidden_layer_sizes=(128, 64), random_state=1)
clf.fit(X, y)
# data=tf.reshape(df,(35007,1713,1)) #reshape data
#print(data)
# conv1d=tf.keras.layers.Conv1D(64,3, activation ='relu', input_shape= (1000, 1)) # the layer accpets any input with shape (*,1). 
# That means the last dimention should be 1
# cnn_data=conv1d(data) # output cnn_data has shape  (9923,998,64)

ValueError: ignored

In [110]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_3 (Conv2D)           (None, 298, 298, 32)      320       
                                                                 
 max_pooling2d_2 (MaxPooling  (None, 149, 149, 32)     0         
 2D)                                                             
                                                                 
 conv2d_4 (Conv2D)           (None, 147, 147, 64)      18496     
                                                                 
 max_pooling2d_3 (MaxPooling  (None, 73, 73, 64)       0         
 2D)                                                             
                                                                 
 conv2d_5 (Conv2D)           (None, 71, 71, 64)        36928     
                                                                 
 flatten_1 (Flatten)         (None, 322624)           

In [111]:
model.fit(X, y, epochs=5, batch_size=32)

ValueError: ignored

In [ ]:
from rdkit import Chem

smiles = 'OC(C(C)=O)=O'
mol = Chem.MolFromSmiles(smiles)
display(mol)   # the molecule can be displayed in a Jupyter Notebook

None

In [6]:
!pip install deepchem

     |████████████████████████████████| 608 kB 25.9 MB/s 
     |████████████████████████████████| 15.7 MB 262 kB/s 
     |████████████████████████████████| 20.6 MB 5.1 MB/s 
  Attempting uninstall: numpy
    Found existing installation: numpy 1.19.5
    Uninstalling numpy-1.19.5:
      Successfully uninstalled numpy-1.19.5
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
yellowbrick 1.3.post1 requires numpy<1.20,>=1.16.0, but you have numpy 1.21.5 which is incompatible.
datascience 0.10.6 requires folium==0.2.1, but you have folium 0.8.3 which is incompatible.
albumentations 0.1.12 requires imgaug<0.2.7,>=0.2.5, but you have imgaug 0.2.9 which is incompatible.


In [6]:
import deepchem as dc
smiles = ['CC(=O)OC1=CC=CC=C1C(=O)O']
featurizer = dc.feat.SmilesToImage(img_size=80, img_spec='std')
images = featurizer.featurize(smiles)
type (images[0])
images[0]

array([[[0.],
        [0.],
        [0.],
        ...,
        [0.],
        [0.],
        [0.]],

       [[0.],
        [0.],
        [0.],
        ...,
        [0.],
        [0.],
        [0.]],

       [[0.],
        [0.],
        [0.],
        ...,
        [0.],
        [0.],
        [0.]],

       ...,

       [[0.],
        [0.],
        [0.],
        ...,
        [0.],
        [0.],
        [0.]],

       [[0.],
        [0.],
        [0.],
        ...,
        [0.],
        [0.],
        [0.]],

       [[0.],
        [0.],
        [0.],
        ...,
        [0.],
        [0.],
        [0.]]])

In [ ]:
from rdkit import Chem
from rdkit.Chem import rdChemReactions

reaction_smiles = 'C=CC1=C(C)c2cc3[nH]c(cc4nc(cc5[nH]c(cc1n2)c(C)c5CCC(C)=O)C(CCC(C)=O)=C4C)c(C)c3C=C.C=CC1=C(C)c2cc3[nH]c(cc4nc(cc5[nH]c(cc1n2)c(C)c5CCC(C)=O)C(CCC(C)=O)=C4C)c(C)c3C=C.O=C1O[C@H]([C@@H](O)CO)[C@@H](O)[C@@H]1O.[Fe+3].[Fe+3]>>C=CC1=C(C)c2cc3[nH]c(cc4nc(cc5[nH]c(cc1n2)c(C)c5CCC(C)=O)C(CCC(C)=O)=C4C)c(C)c3C=C.C=CC1=C(C)c2cc3[nH]c(cc4nc(cc5[nH]c(cc1n2)c(C)c5CCC(C)=O)C(CCC(C)=O)=C4C)c(C)c3C=C.O=C1O[C@H]([C@@H](O)CO)C(O)=C1O.[Fe+2].[Fe+2].[H+].[H+]'
product_smiles = 'C=CC1=C(C)c2cc3[nH]c(cc4nc(cc5[nH]c(cc1n2)c(C)c5CCC(C)=O)C(CCC(C)=O)=C4C)c(C)c3C=C.C=CC1=C(C)c2cc3[nH]c(cc4nc(cc5[nH]c(cc1n2)c(C)c5CCC(C)=O)C(CCC(C)=O)=C4C)c(C)c3C=C.O=C1O[C@H]([C@@H](O)CO)[C@@H](O)[C@@H]1O.[Fe+3].[Fe+3]>>C=CC1=C(C)c2cc3[nH]c(cc4nc(cc5[nH]c(cc1n2)c(C)c5CCC(C)=O)C(CCC(C)=O)=C4C)c(C)c3C=C.C=CC1=C(C)c2cc3[nH]c(cc4nc(cc5[nH]c(cc1n2)c(C)c5CCC(C)=O)C(CCC(C)=O)=C4C)c(C)c3C=C.O=C1O[C@H]([C@@H](O)CO)C(O)=C1O.[Fe+2].[Fe+2].[H+].[H+]'
reactant_smiles = reaction_smiles.split('>>')[0]
reactants = [Chem.MolFromSmiles(x) for x in reactant_smiles.split('.')]
reactant_smarts = '.'.join([Chem.MolToSmarts(x) for x in reactants])

product_smiles = product_smiles.split('>>')[1]
products = [Chem.MolFromSmiles(x) for x in product_smiles.split('.')]
product_smarts = '.'.join([Chem.MolToSmarts(x) for x in products])

reaction_smarts = '{}>>{}'.format(reactant_smarts, product_smarts)
reaction = rdChemReactions.ReactionFromSmarts(reaction_smarts)
display(reaction)  # the reaction can be displayed in a Jupyter Notebook

In [2]:
!pip install transformers

     |████████████████████████████████| 3.4 MB 19.6 MB/s 
     |████████████████████████████████| 596 kB 36.4 MB/s 
     |████████████████████████████████| 67 kB 5.2 MB/s 
     |████████████████████████████████| 3.3 MB 44.4 MB/s 
     |████████████████████████████████| 895 kB 40.2 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [3]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("seyonec/ChemBERTa-zinc-base-v1") 

smiles = 'C=CC1=C(C)c2cc3[nH]c(cc4nc(cc5[nH]c(cc1n2)c(C)c5CCC(C)=O)C(CCC(C)=O)=C4C)c(C)c3C=C.C=CC1=C(C)c2cc3[nH]c(cc4nc(cc5[nH]c(cc1n2)c(C)c5CCC(C)=O)C(CCC(C)=O)=C4C)c(C)c3C=C.O=C1O[C@H]([C@@H](O)CO)[C@@H](O)[C@@H]1O.[Fe+3].[Fe+3]>>C=CC1=C(C)c2cc3[nH]c(cc4nc(cc5[nH]c(cc1n2)c(C)c5CCC(C)=O)C(CCC(C)=O)=C4C)c(C)c3C=C.C=CC1=C(C)c2cc3[nH]c(cc4nc(cc5[nH]c(cc1n2)c(C)c5CCC(C)=O)C(CCC(C)=O)=C4C)c(C)c3C=C.O=C1O[C@H]([C@@H](O)CO)C(O)=C1O.[Fe+2].[Fe+2].[H+].[H+]'
embedding = tokenizer.encode_plus(smiles)

Downloading:   0%|          | 0.00/166 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/501 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/9.21k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/3.14k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/150 [00:00<?, ?B/s]

In [42]:
print(embedding.input_ids)

[0, 39, 33, 262, 21, 33, 39, 12, 39, 13, 71, 22, 261, 23, 63, 290, 65, 71, 12, 261, 24, 270, 12, 261, 25, 63, 290, 65, 71, 12, 261, 21, 82, 22, 13, 71, 12, 39, 13, 71, 25, 271, 12, 39, 295, 51, 13, 39, 12, 271, 12, 39, 295, 51, 295, 39, 24, 39, 13, 71, 12, 39, 13, 71, 23, 39, 33, 39, 18, 39, 33, 262, 21, 33, 39, 12, 39, 13, 71, 22, 261, 23, 63, 290, 65, 71, 12, 261, 24, 270, 12, 261, 25, 63, 290, 65, 71, 12, 261, 21, 82, 22, 13, 71, 12, 39, 13, 71, 25, 271, 12, 39, 295, 51, 13, 39, 12, 271, 12, 39, 295, 51, 295, 39, 24, 39, 13, 71, 12, 39, 13, 71, 23, 39, 33, 39, 18, 51, 33, 39, 21, 51, 63, 39, 36, 44, 318, 39, 266, 44, 265, 51, 13, 275, 272, 39, 266, 44, 265, 51, 272, 39, 266, 44, 65, 21, 51, 18, 63, 42, 73, 15, 23, 65, 18, 63, 42, 73, 15, 23, 65, 34, 34, 39, 33, 262, 21, 33, 39, 12, 39, 13, 71, 22, 261, 23, 63, 290, 65, 71, 12, 261, 24, 270, 12, 261, 25, 63, 290, 65, 71, 12, 261, 21, 82, 22, 13, 71, 12, 39, 13, 71, 25, 271, 12, 39, 295, 51, 13, 39, 12, 271, 12, 39, 295, 51, 295, 39, 